# Day-3 Infographic renderer

Reads `docs/day3_infographic.json` and writes `docs/day3_infographic.png`.

Works in Colab and local dev. No special fonts, no theme packages.

In [ ]:
import json, os, datetime as dt
import matplotlib.pyplot as plt

ROOT = os.path.abspath(os.path.join(os.getcwd(), '..')) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
JSON_PATH = os.environ.get('INFO_JSON', os.path.join(ROOT, 'docs', 'day3_infographic.json'))
with open(JSON_PATH) as f:
    meta = json.load(f)

PNG_PATH = os.environ.get('INFO_PNG', os.path.join(ROOT, 'docs', 'day3_infographic.png'))

if meta.get('timestamp') == '{{AUTO}}':
    meta['timestamp'] = dt.datetime.utcnow().isoformat(timespec='seconds') + 'Z'

title = meta.get('title', 'Infographic')
subtitle = meta.get('subtitle', '')
m = meta.get('metrics', {})
notes = meta.get('notes', [])
visuals = meta.get('visuals', {})
palette = visuals.get('palette', [])
accent = palette[0] if palette else None
bg = palette[1] if len(palette) > 1 else None

# --- Layout ---------------------------------------------------------------
fig = plt.figure(figsize=(10, 6), dpi=160)
ax = plt.gca()
ax.axis('off')

if bg:
    fig.patch.set_facecolor(bg)
    ax.set_facecolor(bg)

def T(x, size=14, weight='normal', color=None, y=None):
    return ax.text(0.05, y, x, transform=ax.transAxes, fontsize=size,
                   fontweight=weight, color=color)


T(title, size=20, weight='bold', color=accent, y=0.92)
T(subtitle, size=12, color=(0.8,0.8,0.85), y=0.87)

lines = [
f"NP-wall rate: {m.get('np_wall_pct', 0):.2f}%",
f"ΔΦ threshold (NP): {m.get('np_threshold','?')}",
f"ΔΦ threshold (P):  {m.get('p_threshold','?')}",
f"Samples × length:  {m.get('samples','?')} × {m.get('trace_len','?')}"
]
y = 0.75
for s in lines:
    T(s, size=14, color='white', y=y)
    y -= 0.06

T('Notes:', size=14, weight='bold', color=accent, y=y - 0.02)
y -= 0.10
for h in notes:
    T('• ' + h, size=13, color='white', y=y)
    y -= 0.05

T(f"Created: {meta['timestamp']}", size=9, color=(0.7,0.72,0.8), y=0.06)

os.makedirs(os.path.dirname(PNG_PATH), exist_ok=True)
plt.savefig(PNG_PATH, bbox_inches='tight')
print(f'Saved → {PNG_PATH}')

In [ ]:
# Optional: quick view in notebook
from IPython.display import Image, display
display(Image(filename=PNG_PATH))

## Notes
- Edit numbers in `docs/day3_infographic.json` or point `INFO_JSON` env to another file.
- CI can set INFO_JSON/INFO_PNG to render alternative outputs.